# Summarization with blurr
> blurr is a libray I started that integrates huggingface transformers with the world of fastai v2, giving fastai devs everything they need to train, evaluate, and deploy transformer specific models.  In this article, I provide a simple example of how to use blurr's new summarization capabilities to train, evaluate, and deploy a BART summarization model.

* Updated on 08/21/2020 to use fastai 2.0.0 and also demo batch-time padding.  
* Updated on 09/25/2020 to use on the fly batch-time tokenization.  
* Updated on 11/12/2020 with support for fastai >= 2.1.5 and mixed precision.  

- author: Wayde Gilliam

### Note

1. The blurr library will break under transformer 4.0.X. Stick with 3.5.1 for the moment. See [the issue](https://github.com/ohmeow/blurr/issues/21).
2. Apex seems to be causing problem. The solution is [reinstall it](https://github.com/NVIDIA/apex#quick-start). (Python only build) 

```
/usr/local/lib/python3.6/dist-packages/apex/normalization/fused_layer_norm.py in forward(ctx, input, weight, bias, normalized_shape, eps)
     23     bias_ = bias.contiguous()
     24     output, mean, invvar = fused_layer_norm_cuda.forward_affine(
---> 25         input_, ctx.normalized_shape, weight_, bias_, ctx.eps)
     26     ctx.save_for_backward(input_, weight_, bias_, mean, invvar)
     27     return output

RuntimeError: Unrecognized tensor type ID: AutogradCUDA
```

In [1]:
# only run this cell if you are in collab
# !pip install ohmeow-blurr -q
# !pip install datasets -q

In [1]:
import datasets
import pandas as pd
from fastai.text.all import *
from transformers import *

from blurr.data.all import *
from blurr.modeling.all import *

## Data Preparation

We're going to use to use the new [nlp](https://github.com/huggingface/nlp) library from huggingface to grab your raw data.  This package gives you access to all kinds of NLP related datasets, explanations of each, and various task specific metrics to use in evaluating your model.  The best part being everything comes down to you in JSON!  This makes it a breeze to get up and running quickly!

We'll just use a subset of the training set to build both our training and validation DataLoaders

In [2]:
raw_data = datasets.load_dataset('cnn_dailymail', '3.0.0', split='train[:1%]')

Reusing dataset cnn_dailymail (/root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/0128610a44e10f25b4af6689441c72af86205282d26399642f7db38fa7535602)


In [3]:
df = pd.DataFrame(raw_data)
df.head(2)

,article,highlights,id
0,"It's official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria. Obama sent a letter to the heads of the House and Senate on Saturday night, hours after announcing that he believes military action against Syrian targets is the right step to take over the alleged use of chemical weapons. The proposed legislation from Obama asks Congress to approve the use of military force ""to deter, disrupt, prevent and degrade the potential for future uses of chemical weapons or other weapons of mass destruction."" It's a step that is set to turn an internat...","Syrian official: Obama climbed to the top of the tree, ""doesn't know how to get down""\nObama sends a letter to the heads of the House and Senate .\nObama to seek congressional approval on military action against Syria .\nAim is to determine whether CW were used, not by whom, says U.N. spokesman .",0001d1afc246a7964130f43ae940af6bc6c57f01
1,"(CNN) -- Usain Bolt rounded off the world championships Sunday by claiming his third gold in Moscow as he anchored Jamaica to victory in the men's 4x100m relay. The fastest man in the world charged clear of United States rival Justin Gatlin as the Jamaican quartet of Nesta Carter, Kemar Bailey-Cole, Nickel Ashmeade and Bolt won in 37.36 seconds. The U.S finished second in 37.56 seconds with Canada taking the bronze after Britain were disqualified for a faulty handover. The 26-year-old Bolt has now collected eight gold medals at world championships, equaling the record held by American trio...",Usain Bolt wins third gold of world championship .\nAnchors Jamaica to 4x100m relay victory .\nEighth gold at the championships for Bolt .\nJamaica double up in women's 4x100m relay .,0002095e55fcbd3a2f366d9bf92a95433dc305ef


We begin by getting our hugginface objects needed for this task (e.g., the architecture, tokenizer, config, and model).  We'll use blurr's `get_hf_objects` helper method here.

In [4]:
pretrained_model_name = "facebook/bart-large-cnn"
hf_arch, hf_config, hf_tokenizer, hf_model = BLURR_MODEL_HELPER.get_hf_objects(pretrained_model_name, 
                                                                               model_cls=BartForConditionalGeneration)

# BLURR_MODEL_HELPER.get_hf_objects??
hf_arch, type(hf_config), type(hf_tokenizer), type(hf_model)

('bart',
 transformers.configuration_bart.BartConfig,
 transformers.tokenization_bart.BartTokenizer,
 transformers.modeling_bart.BartForConditionalGeneration)

Next we need to build out our DataBlock.  Remember tha a DataBlock is a blueprint describing how to move your raw data into something modelable.  That blueprint is executed when we pass it a data source, which in our case, will be the DataFrame we created above. We'll use a random subset to get things moving along a bit faster for the demo as well.

Notice that the blurr DataBlock as been dramatically simplified given the shift to on-the-fly batch-time tokenization.  All we need is to define a single `HF_SummarizationBeforeBatchTransform` instance, optionally passing a list to any of the tokenization arguments to differentiate the values for the input and summary sequences.  We pass `noop` as a type transform for our targets because everything is already handled by the batch transform now.

In [5]:
hf_batch_tfm = HF_SummarizationBeforeBatchTransform(hf_arch, hf_tokenizer, max_length=[256, 130])

blocks = (HF_TextBlock(before_batch_tfms=hf_batch_tfm, input_return_type=HF_SummarizationInput), noop)

dblock = DataBlock(blocks=blocks, 
                   get_x=ColReader('article'), 
                   get_y=ColReader('highlights'), 
                   splitter=RandomSplitter())

In [6]:
dls = dblock.dataloaders(df, bs=16)

In [7]:
len(dls.train.items), len(dls.valid.items)

(2297, 574)

It's always a good idea to check out a batch of data and make sure the shapes look right.

In [8]:
b = dls.one_batch()
len(b), b[0]['input_ids'].shape, b[1].shape

(2, torch.Size([16, 256]), torch.Size([16, 76]))

Even better, we can take advantage of blurr's TypeDispatched version of `show_batch` to look at things a bit more intuitively.  We pass in the `dls` via the `dataloaders` argument so we can access all tokenization/modeling configuration stored in our batch transform above.

In [9]:
dls.show_batch(dataloaders=dls, max_n=2)

,text,target
0,"While Iraq's military claimed Wednesday to have driven back militants battling for control of the country, the chairman of the Joint Chiefs of Staff told Congress that the United States has received a request from the Iraqi government to use its air power in the conflict. Gen. Martin Dempsey, the senior ranking member of the U.S. armed forces, spoke before the Senate Appropriations Committee Wednesday on Capitol Hill in Washington, saying that the United States' ""national security interest (is) to counter (ISIS) where we find them."" ISIS is the Islamic State in Iraq and Syria. Comprising mostly Sunni Muslims, ISIS is an al Qaeda splinter group that wants to establish a caliphate, or Islamic state, that would stretch from Iraq into northern Syria. The group has had substantial success in Syria battling Syrian President Bashar al-Assad's security forces. Since launching their offensive in Iraq, ISIS claims to have killed at least 1,700 Shiites. Hundreds of thousands of Iraqis have fled, prompting fears of a brewing humanitarian crisis. Qassim Atta, a spokesman for Iraqi security forces, on Wednesday night said an investigation had been ordered into 59 high-ranking security officials accused of leaving their posts. The officials could be executed if found guilty, Atta said. Concerns over","Vice President Biden stresses need for national unity in talk with Iraqi PM.\nA cleric called for attacks against U.S. embassies in the case of airstrikes.\nInvestigation will probe Iraqi security forces who left posts, general commander says.\nSaudi Arabia responds to Iraq's accusation that it's helping ISIS, calling allegation a ""falsehood"""
1,"Washington (CNN) -- Congressional negotiators reached a tentative deal Tuesday to extend the payroll tax cut and unemployment benefits while avoiding a fee cut for Medicare doctors, according to Republican legislators and aides. The framework deal followed a key Republican concession Monday and could receive the endorsement of a House-Senate conference committee on Wednesday. Reps. Renee Ellmers of North Carolina and Fred Upton of Michigan, both Republicans who were part of the conference committee negotiations, called it a tentative agreement, while a top House Democratic aide said Democratic leaders would discuss the proposal with their members on Wednesday morning. Ellmers and Upton said they expected the conference committee to sign off on the measure Wednesday if no objections arise. In that case, the House would vote on the agreement, in the form of the conference committee report, as soon as Friday. However, some conservative House Republicans expressed concerns about the potential agreement and said they were unsure if they would support it. Other GOP House legislators said they expected it to pass with support from a majority of Republicans, as well as Democrats. ""It's the art of a deal. I mean, it's a compromise,"" said Rep. Steve Latourette, R-Ohio. ""So you have people that didn't get a 100% of what they wanted."" Under",NEW: Some Republican congressmen question the tentative agreement.\nProgress is made after Republicans back down on payroll tax cut.\nPolitical analyst Norm Ornstein warns new GOP strategy could backfire.\nPresident Obama warns that failing to act could hurt the economic recovery.


## Training

We'll prepare our BART model for training by wrapping it in blurr's `HF_BaseModelWrapper` model object and defining a new callback, `HF_SummarizationModelCallback`.  This class will handle ensuring all our inputs get translated into the proper arguments needed by a huggingface conditional generation model.  We'll also use a custom model splitter that will allow us to apply discriminative learning rates over the various layers in our huggingface model.

Once we have everything in place, we'll freeze our model so that only the last layer group's parameters of trainable.  See [here](https://docs.fast.ai/basic_train.html#Discriminative-layer-training) for our discriminitative learning rates work in fastai.

**Note:** This has been tested with BART only thus far (if you try any other conditional generation transformer models they may or may not work ... if you do, lmk either way)

In [10]:
text_gen_kwargs = { **hf_config.task_specific_params['summarization'], **{'max_length': 130, 'min_length': 30} }
text_gen_kwargs

{'early_stopping': True,
 'length_penalty': 2.0,
 'max_length': 130,
 'min_length': 30,
 'no_repeat_ngram_size': 3,
 'num_beams': 4}

In [11]:
model = HF_BaseModelWrapper(hf_model)
model_cb = HF_SummarizationModelCallback(text_gen_kwargs=text_gen_kwargs)

learn = Learner(dls, 
                model,
                opt_func=ranger,
                loss_func=CrossEntropyLossFlat(),
                cbs=[model_cb],
                splitter=partial(summarization_splitter, arch=hf_arch)).to_fp16()

learn.create_opt() 
learn.freeze()

Still experimenting with how to use fastai's learning rate finder for these kinds of models.  If you all have any suggestions or interesting insights to share, please let me know.  We're only going to train the frozen model for one epoch for this demo, but feel free to progressively unfreeze the model and train the other layers to see if you can best my results below.

/usr/local/lib/python3.6/dist-packages/fastai/learner.py:53: UserWarning: Could not load the optimizer state.
  if with_opt: warn("Could not load the optimizer state.")


SuggestedLRs(lr_min=0.09120108485221863, lr_steep=0.7585775852203369)

In [12]:
learn.lr_find(suggestions=True)

It's also not a bad idea to run a batch through your model and make sure the shape of what goes in, and comes out, looks right.

In [14]:
b = dls.one_batch()
preds = learn.model(b[0])
len(preds),preds[0], preds[1].shape

(3,
 tensor(4.4479, device='cuda:0', grad_fn=<NllLossBackward>),
 torch.Size([16, 82, 50264]))

In [13]:
learn.fit_one_cycle(1, lr_max=3e-5)

epoch,train_loss,valid_loss,rouge1,rouge2,rougeL,time
0,2.039902,1.835553,0.377645,0.159232,0.260118,06:06


In [14]:
# reduce the batch size after unfreezing
learn.unfreeze()
learn.dls.bs = 2

SuggestedLRs(lr_min=4.786300996784121e-05, lr_steep=2.75422871709452e-06)

In [15]:
learn.lr_find(suggestions=True)

In [ ]:
learn.fit_one_cycle(3, lr_max=slice(3e-6, 3e-5))

epoch,train_loss,valid_loss,rouge1,rouge2,rougeL,time


And now we can look at the "greedy decoded" predictions ...

In [ ]:
learn.show_results(learner=learn, max_n=2)

Even better though, blurr augments the fastai Learner with a `blurr_summarize` method that allows you to use huggingface's `PreTrainedModel.generate` method to create something more human-like.

In [ ]:
test_article = """
The past 12 months have been the worst for aviation fatalities so far this decade - with the total of number of people killed if airline crashes reaching 1,050 even before the Air Asia plane vanished. Two incidents involving Malaysia Airlines planes - one over eastern Ukraine and the other in the Indian Ocean - led to the deaths of 537 people, while an Air Algerie crash in Mali killed 116 and TransAsia Airways crash in Taiwan killed a further 49 people. The remaining 456 fatalities were largely in incidents involving small commercial planes or private aircraft operating on behalf of companies, governments or organisations. Despite 2014 having the highest number of fatalities so far this decade, the total number of crashes was in fact the lowest since the first commercial jet airliner took off in 1949 - totalling just 111 across the whole world over the past 12 months. The all-time deadliest year for aviation was 1972 when a staggering 2,429 people were killed in a total of 55 plane crashes - including the crash of Aeroflot Flight 217, which killed 174 people in Russia, and Convair 990 Coronado, which claimed 155 lives in Spain. However this year's total death count of 1,212, including those presumed dead on board the missing Air Asia flight, marks a significant rise on the very low 265 fatalities in 2013 - which led to it being named the safest year in aviation since the end of the Second World War. Scroll down for videos. Deadly: The past 12 months have been the worst for aviation fatalities so far this decade - with the total of number of people killed if airline crashes reaching 1,158 even before the Air Asia plane (pictured) vanished. Fatal: Two incidents involving Malaysia Airlines planes - one over eastern Ukraine (pictured) and the other in the Indian Ocean - led to the deaths of 537 people. Surprising: Despite 2014 having the highest number of fatalities so far this decade, the total number of crashes was in fact the lowest since the first commercial jet airliner took off in 1949. 2014 has been a horrific year for Malaysia-based airlines, with 537 people dying on Malaysia Airlines planes, and a further 162 people missing and feared dead in this week's Air Asia incident. In total more than half the people killed in aviation incidents this year had been flying on board Malaysia-registered planes. In January a total of 12 people lost their lives in five separate incidents, while the same number of crashes in February killed 107. 
"""

In [ ]:
outputs = learn.blurr_summarize(test_article, early_stopping=True, num_beams=4, num_return_sequences=3)

for idx, o in enumerate(outputs):
    print(f'=== Prediction {idx+1} ===\n{o}\n')

What about inference?  Easy!

In [ ]:
learn.export(fname='/storage/ft_cnndm_export.pkl')

In [ ]:
inf_learn = load_learner(fname='/storage/ft_cnndm_export.pkl')
inf_learn.blurr_summarize(test_article)

## That's it

[blurr](https://ohmeow.github.io/blurr/) supports a number of huggingface transformer model tasks in addition to summarization (e.g., sequence classification , token classification, and question/answering). The docs include examples for each of these tasks if you're curious to learn more.

For more information about ohmeow or to get in contact with me, head over to [ohmeow.com](ohmeow.com) for all the details.

Thanks!